### Creating your own dataset

In [3]:
import requests

url = "https://api.github.com/repos/huggingface/datasets/issues?page=1&per_page=1"
response = requests.get(url)

response.status_code

200

In [4]:
response.json()

[{'url': 'https://api.github.com/repos/huggingface/datasets/issues/7999',
  'repository_url': 'https://api.github.com/repos/huggingface/datasets',
  'labels_url': 'https://api.github.com/repos/huggingface/datasets/issues/7999/labels{/name}',
  'comments_url': 'https://api.github.com/repos/huggingface/datasets/issues/7999/comments',
  'events_url': 'https://api.github.com/repos/huggingface/datasets/issues/7999/events',
  'html_url': 'https://github.com/huggingface/datasets/issues/7999',
  'id': 3915367642,
  'node_id': 'I_kwDODunzps7pX8Ta',
  'number': 7999,
  'title': 'Too many dataloader workers: 4 (max is dataset.num_shards=3). Stopping 1 dataloader workers.',
  'user': {'login': 'D222097',
   'id': 50061868,
   'node_id': 'MDQ6VXNlcjUwMDYxODY4',
   'avatar_url': 'https://avatars.githubusercontent.com/u/50061868?v=4',
   'gravatar_id': '',
   'url': 'https://api.github.com/users/D222097',
   'html_url': 'https://github.com/D222097',
   'followers_url': 'https://api.github.com/users/D

In [ ]:
# GITHUB_TOKEN = ""
# headers = {"Authorization": f"token {GITHUB_TOKEN}"}

In [19]:
import time
import math
from pathlib import Path
import pandas as pd
from tqdm.notebook import tqdm

def fetch_issues(
    owner="huggingface",
    repo="datasets",
    num_issues=5_000,
    rate_limit=5_5000,
    issues_path=Path("Data/")
):
    if not issues_path.is_dir():
        issues_path.mkdir(exist_ok=True)
    
    batch = []
    all_issues = []
    per_page = 100
    num_pages = math.ceil(num_issues/per_page)
    base_url = "https://api.github.com/repos"

    for page in tqdm(range(num_pages)):
        query = f"issues?page={page}&per_page={per_page}&state=all"
        issues = requests.get(f"{base_url}/{owner}/{repo}/{query}", headers=headers)
        batch.extend(issues.json())

        if len(batch) > rate_limit and len(all_issues) < num_issues:
            all_issues.extend(batch)
            batch = []
            print(f"Reached Github rate limit. Sleeping for one hour ... ")
            time.sleep(60*60+1)

    all_issues.extend(batch)
    df = pd.DataFrame.from_records(all_issues)
    df.to_json(f"{issues_path}/{repo}-issues.jsonl", orient="records", lines=True)
    print(
        f"Downloaded all the issues for {repo}! Dataset stored at {issues_path}/{repo}-issues.jsonl"
    )

In [20]:
fetch_issues()

  0%|          | 0/50 [00:00<?, ?it/s]

Downloaded all the issues for datasets! Dataset stored at Data/datasets-issues.jsonl


In [49]:
# load issues using load dataset
from datasets import load_dataset

issues_dataset = load_dataset("json", data_files="Data/datasets-issues.jsonl", split="train")
issues_dataset

Dataset({
    features: ['url', 'repository_url', 'labels_url', 'comments_url', 'events_url', 'html_url', 'id', 'node_id', 'number', 'title', 'user', 'labels', 'state', 'locked', 'assignee', 'assignees', 'milestone', 'comments', 'created_at', 'updated_at', 'closed_at', 'author_association', 'type', 'active_lock_reason', 'sub_issues_summary', 'issue_dependencies_summary', 'body', 'closed_by', 'reactions', 'timeline_url', 'performed_via_github_app', 'state_reason', 'draft', 'pull_request'],
    num_rows: 5000
})

## Cleaning up the data

Since the contents of issues and pull requests are quite different, let’s do some minor preprocessing to enable us to distinguish between them.

In [50]:
sample = issues_dataset.shuffle(seed=666).select(range(3))

for url, pr in zip(sample["html_url"], sample["pull_request"]):
    print(f">> URL: {url}")
    print(f">> PR: {pr}")

>> URL: https://github.com/huggingface/datasets/issues/7281
>> PR: None
>> URL: https://github.com/huggingface/datasets/pull/4724
>> PR: {'url': 'https://api.github.com/repos/huggingface/datasets/pulls/4724', 'html_url': 'https://github.com/huggingface/datasets/pull/4724', 'diff_url': 'https://github.com/huggingface/datasets/pull/4724.diff', 'patch_url': 'https://github.com/huggingface/datasets/pull/4724.patch', 'merged_at': datetime.datetime(2022, 9, 5, 17, 25, 27)}
>> URL: https://github.com/huggingface/datasets/pull/7120
>> PR: {'url': 'https://api.github.com/repos/huggingface/datasets/pulls/7120', 'html_url': 'https://github.com/huggingface/datasets/pull/7120', 'diff_url': 'https://github.com/huggingface/datasets/pull/7120.diff', 'patch_url': 'https://github.com/huggingface/datasets/pull/7120.patch', 'merged_at': datetime.datetime(2024, 8, 22, 14, 33, 52)}


Here we can see that each pull request is associated with various URLs, while ordinary issues have a None entry. We can use this distinction to create a new is_pull_request column that checks whether the pull_request field is None or not:

In [51]:
issues_dataset = issues_dataset.map((lambda x: {"is_pull_request": False if  x["pull_request"] is None else True}))

### Augumenting the dataset
the comments associated with an issue or pull request provide a rich source of information, especially if we’re interested in building a search engine to answer user queries about the library

In [56]:
issue_number = 2792
url = f"https://api.github.com/repos/huggingface/datasets/issues/{issue_number}/comments"
response = requests.get(url, headers=headers)
response.json()

[{'url': 'https://api.github.com/repos/huggingface/datasets/issues/comments/897594128',
  'html_url': 'https://github.com/huggingface/datasets/pull/2792#issuecomment-897594128',
  'issue_url': 'https://api.github.com/repos/huggingface/datasets/issues/2792',
  'id': 897594128,
  'node_id': 'IC_kwDODunzps41gDMQ',
  'user': {'login': 'bhavitvyamalik',
   'id': 19718818,
   'node_id': 'MDQ6VXNlcjE5NzE4ODE4',
   'avatar_url': 'https://avatars.githubusercontent.com/u/19718818?v=4',
   'gravatar_id': '',
   'url': 'https://api.github.com/users/bhavitvyamalik',
   'html_url': 'https://github.com/bhavitvyamalik',
   'followers_url': 'https://api.github.com/users/bhavitvyamalik/followers',
   'following_url': 'https://api.github.com/users/bhavitvyamalik/following{/other_user}',
   'gists_url': 'https://api.github.com/users/bhavitvyamalik/gists{/gist_id}',
   'starred_url': 'https://api.github.com/users/bhavitvyamalik/starred{/owner}{/repo}',
   'subscriptions_url': 'https://api.github.com/users/

We can see that the comment is stored in the body field, so let’s write a simple function that returns all the comments associated with an issue by picking out the body contents for each element in response.json():

In [68]:
def get_comments(issue_number):
    
    url = f"https://api.github.com/repos/huggingface/datasets/issues/{issue_number}/comments"
    response = requests.get(url, headers=headers)
    
    # Check if the request was successful
    if response.status_code != 200:
        return []
    
    # Parse JSON response
    data = response.json()
    
    # Ensure data is a list before trying to iterate
    if not isinstance(data, list):
        return []
    
    # Extract comments with additional safety check
    return [r["body"] for r in data if isinstance(r, dict) and "body" in r]


# Test our function works as expected
get_comments(2792)

["@albertvillanova my tests are failing here:\r\n```\r\ndataset_name = 'gooaq'\r\n\r\n    def test_load_dataset(self, dataset_name):\r\n        configs = self.dataset_tester.load_all_configs(dataset_name, is_local=True)[:1]\r\n>       self.dataset_tester.check_load_dataset(dataset_name, configs, is_local=True, use_local_dummy_data=True)\r\n\r\ntests/test_dataset_common.py:234: \r\n_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ \r\ntests/test_dataset_common.py:187: in check_load_dataset\r\n    self.parent.assertTrue(len(dataset[split]) > 0)\r\nE   AssertionError: False is not true\r\n```\r\nWhen I try loading dataset on local machine it works fine. Any suggestions on how can I avoid this error?",
 'Thanks for the help, @albertvillanova! All tests are passing now.']

 so let’s use Dataset.map() to add a new comments column to each issue in our dataset:

In [69]:
issues_with_comments_dataset = issues_dataset.map(
    lambda x: {"comments": get_comments(x["number"])}
)

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

In [70]:
issues_with_comments_dataset.push_to_hub("github-issues")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ? shards/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Processing Files (0 / 0): |          |  0.00B /  0.00B            

New Data Upload: |          |  0.00B /  0.00B            

CommitInfo(commit_url='https://huggingface.co/datasets/DrSly/github-issues/commit/0c2874fb4afc42bf8177c76919713a8a9cd54b2f', commit_message='Upload dataset', commit_description='', oid='0c2874fb4afc42bf8177c76919713a8a9cd54b2f', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/DrSly/github-issues', endpoint='https://huggingface.co', repo_type='dataset', repo_id='DrSly/github-issues'), pr_revision=None, pr_num=None)

In [71]:
# Load dataset
remote_dataset = load_dataset("DrSly/github-issues", split="train")
remote_dataset

README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/9.71M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/5000 [00:00<?, ? examples/s]

Dataset({
    features: ['url', 'repository_url', 'labels_url', 'comments_url', 'events_url', 'html_url', 'id', 'node_id', 'number', 'title', 'user', 'labels', 'state', 'locked', 'assignee', 'assignees', 'milestone', 'comments', 'created_at', 'updated_at', 'closed_at', 'author_association', 'type', 'active_lock_reason', 'sub_issues_summary', 'issue_dependencies_summary', 'body', 'closed_by', 'reactions', 'timeline_url', 'performed_via_github_app', 'state_reason', 'draft', 'pull_request', 'is_pull_request'],
    num_rows: 5000
})